In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import joblib
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from torch.amp import autocast, GradScaler
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils import clip_grad_norm_
import os
import sys

In [ ]:
# Add the parent directory (where utils.py is located) to sys.path to ensure utils module is found
utils_path = os.path.abspath(os.path.join(os.getcwd(), "../../Utils/"))
sys.path.append(utils_path)

from model_utils import ImprovedTagClassifier  # Import our model from utils

In [ ]:
# Enable cuDNN benchmarking for faster training
torch.backends.cudnn.benchmark = True

In [ ]:
# 1. Load dataset and remove rows with '-' in the tag column
df = pd.read_csv("../Dataset/figma_dataset.csv")

# Remove rows with weird tags (like '-' or specific unwanted ones)
df = df[~df['tag'].str.contains(r'[-:]', regex=True)]
df = df[~df['tag'].str.contains(r'\b(CNX|ADDRESS|ASIDE|CANVAS|CITE|DD|DL|DT|ICON|S|VECTOR|DEL|LEGEND|BDI|LOGO|OBJECT|OPTGROUP|CENTER|CODE|BLOCKQUOTE|FRONT|Q|IFRAME|A|HR|SEARCH|DETAILS|FIELDSET|SLOT|SVG|AD|ADSLOT|AUDIO|BLINK|BOLD|COL|COLGROUP|COMMENTS|DATA|DIALOG|EMBED|EMPHASIS|FONT|H7|HGROUP|INS|INTERACTION|ITALIC|ITEMTEMPLATE|MARK|MATH|MENU|MI|MN|MO|MROW|MSUP|NOBR|OFFER|OPTION|PATH|PROGRESS|STRIKE|SWAL|TEXT|TFOOT|TITLE|TT|VAR|VEV|W|WBR|COUNTRY|ESI:INCLUDE|HTTPS:|LOGIN|NOCSRIPT|PERSONAL|STONG|CONTENT|DELIVERY|LEFT|MSUBSUP|KBD|ROOT|PARAGRAPH|BE|AI2SVELTEWRAP|BANNER|PHOTO1|UNK)\b', regex=True)]

# Pattern to spot unwanted tags in other columns
pattern = r'[-:]|\b(CNX|ADDRESS|ASIDE|CANVAS|CITE|DD|DL|DT|ICON|S|VECTOR|DEL|LEGEND|BDI|LOGO|OBJECT|OPTGROUP|CENTER|CODE|BLOCKQUOTE|FRONT|Q|IFRAME|SEARCH|DETAILS|FIELDSET|SLOT|AD|ADSLOT|AUDIO|BLINK|BOLD|COL|COLGROUP|COMMENTS|DATA|DIALOG|EMBED|EMPHASIS|FONT|H7|HGROUP|INS|INTERACTION|ITALIC|ITEMTEMPLATE|MARK|MATH|MENU|MI|MN|MO|MROW|MSUP|NOBR|OFFER|OPTION|PATH|PROGRESS|STRIKE|SWAL|TEXT|TFOOT|TITLE|TT|VAR|VEV|W|WBR|COUNTRY|ESI:INCLUDE|HTTPS:|LOGIN|NOCSRIPT|PERSONAL|STONG|CONTENT|DELIVERY|LEFT|MSUBSUP|KBD|ROOT|PARAGRAPH|BE|AI2SVELTEWRAP|BANNER|PHOTO1|UNK)\b'

# Replace unwanted tags with 'DIV' in related columns
for col in ['prev_sibling_html_tag', 'child_1_html_tag', 'child_2_html_tag']:
    df[col] = np.where(df[col].str.contains(pattern, regex=True, na=False), 'DIV', df[col])

# Map similar tags to simpler ones (e.g., 'H1' to 'P')
tag_mapping = {
    "ARTICLE": "DIV", "DIV": "DIV", "FIGURE": "DIV", "FOOTER": "DIV", "HEADER": "DIV", "NAV": "DIV", "MAIN": "DIV",
    "BODY": "DIV", "FORM": "DIV", "OL": "DIV", "UL": "DIV", "TABLE": "DIV", "THEAD": "DIV", "TBODY": "DIV", "SECTION": "DIV",
    "H1": "P", "H2": "P", "H3": "P", "H4": "P", "H5": "P", "H6": "P", "SUP": "P", "SUB": "P", "BIG": "P",
    "P": "P", "CAPTION": "P", "FIGCAPTION": "P", "B": "P", "EM": "P", "I": "P", "TD": "P", "TH": "P", "TR": "P", "PRE": "P",
    "U": "P", "TIME": "P", "TXT": "P", "ABBR": "P", "SMALL": "P", "STRONG": "P", "SUMMARY": "P", "SPAN": "P", "LABEL": "P", "LI": "P",
    "PICTURE": "IMG", "VIDEO": "IMG",
    "SELECT": "INPUT", "TEXTAREA": "INPUT",
    "VECTOR": "SVG", "ICON": "SVG",
    "UNK": "CONTAINER"
}

# Fix some specific tags based on conditions
df.loc[(df["tag"] == "SPAN") & ((df["type"] == "RECTANGLE") | (df["type"] == "GROUP")), "tag"] = "DIV"

# Replace '-' in child tags with 'DIV'
children_cols = ['child_1_html_tag', 'child_2_html_tag']
for col in children_cols:
    df[col] = df[col].apply(lambda x: "DIV" if isinstance(x, str) and '-' in x else x)

# Make all tags uppercase for consistency
df['tag'] = df['tag'].str.upper()
df['prev_sibling_html_tag'] = df['prev_sibling_html_tag'].str.upper()
df['child_1_html_tag'] = df['child_1_html_tag'].str.upper()
df['child_2_html_tag'] = df['child_2_html_tag'].str.upper()

# Apply the tag mapping to all relevant columns
df['tag'] = df['tag'].replace(tag_mapping)
df['prev_sibling_html_tag'] = df['prev_sibling_html_tag'].replace(tag_mapping)
df['child_1_html_tag'] = df['child_1_html_tag'].replace(tag_mapping)
df['child_2_html_tag'] = df['child_2_html_tag'].replace(tag_mapping)

# Remove some tags we don’t want
df = df[~df['tag'].str.contains(r'\b(P|IMG|CHECKBOX|RADIO)\b', regex=True)]

# Show some info about the cleaned data
print(df['tag'].unique())
print(df.count())
print("Unique Child 1:", df['child_1_html_tag'].unique())
print("Unique Child 2:", df['child_2_html_tag'].unique())
print("Prev HTML tag:", df['prev_sibling_html_tag'].unique())

In [ ]:
# Split the data into features (X) and target (y)
y = df["tag"]
X = df.drop(columns=["tag"])

In [ ]:
# Identify categorical and continuous columns
categorical_cols = ['type','prev_sibling_html_tag','child_1_html_tag','child_2_html_tag']
continuous_cols = [col for col in X.columns if col not in categorical_cols]

In [ ]:
# Process categorical features with OneHotEncoder
X[categorical_cols] = X[categorical_cols].astype(str).fillna('unknown')
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_cat_encoded = ohe.fit_transform(X[categorical_cols])
joblib.dump(ohe, "../models/ohe_encoder.pkl")

In [ ]:
# Better missing value handling with imputer
imputer = SimpleImputer(strategy='most_frequent')
X_continuous_imputed = imputer.fit_transform(X[continuous_cols])
joblib.dump(imputer, "../models/imputer.pkl")

In [ ]:
# Scale continuous features
scaler = StandardScaler()
X_continuous_scaled = scaler.fit_transform(X_continuous_imputed)
joblib.dump(scaler, "../models/scaler.pkl")

In [ ]:
# Combine one-hot encoded categorical features with scaled continuous features
X_processed = np.concatenate([X_cat_encoded, X_continuous_scaled], axis=1)

In [ ]:
# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
joblib.dump(label_encoder, "../models/label_encoder.pkl")

In [ ]:
from collections import Counter

# Count occurrences of each class
class_counts = Counter(y_encoded)

# Find classes with only 1 sample
rare_classes = [cls for cls, count in class_counts.items() if count < 2]

# Duplicate rare class samples
for cls in rare_classes:
    idx = np.where(y_encoded == cls)[0][0]  # Get the index of the rare sample
    original_class_name = label_encoder.inverse_transform([cls])[0]  # Convert back to original label
    print(f"Duplicating class '{original_class_name}' (only 1 sample present).")

    X_processed = np.vstack([X_processed, X_processed[idx]])  # Duplicate features
    y_encoded = np.append(y_encoded, y_encoded[idx])  # Duplicate label

In [ ]:
# Train/test split => remove stratification if there are classes with too few samples
unique_counts = np.unique(y_encoded, return_counts=True)
min_samples = min(unique_counts[1])

if min_samples < 2:
    print(f"Warning: The least populated class has only {min_samples} sample(s). Removing stratification.")
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y_encoded, test_size=0.2, random_state=42
    )
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Convert data to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [ ]:
# Create dataset and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(
    train_dataset, 
    batch_size=512,  # Larger batch size
    shuffle=True, 
    num_workers=8,   # Parallel loading
    pin_memory=True,  # Faster data transfer to GPU
    prefetch_factor=2
)

In [ ]:
# Compute class weights
print("Computing class weights...")
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
for c in np.unique(y_train):
    print(label_encoder.inverse_transform([c])[0])
print(f"Class weights: {class_weights}")
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

In [ ]:
# Initialize model
print("Initializing model...")
input_size = X_train.shape[1]
output_size = len(label_encoder.classes_)
model = ImprovedTagClassifier(input_size, output_size).to(device)

In [ ]:
# 7. Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

In [ ]:
# 8. Setup mixed precision training
scaler = GradScaler(device='cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 9. Training loop with timing and early stopping
print("Starting training...")
best_loss = float('inf')
patience = 10
counter = 0
early_stop = False
start_time = time.time()

num_epochs = 100
for epoch in range(num_epochs):
    epoch_start = time.time()
    model.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        # Move batch to device
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        
        # Use mixed precision for faster training
        with torch.amp.autocast('cuda', enabled=device.type=='cuda'):
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
        
        # Scale gradients and optimize
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    scheduler.step()
    
    epoch_time = time.time() - epoch_start
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}, Time: {epoch_time:.2f}s")
    
    # Early stopping
    if avg_loss < best_loss:
        best_loss = avg_loss
        counter = 0
        torch.save(model.state_dict(), "../models/best_tag_classifier.pth")
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            early_stop = True
    
    if early_stop:
        break

In [ ]:
# Save the trained model
torch.save(model.state_dict(), "../models/tag_classifier.pth")
total_time = time.time() - start_time
print(f"Total training time: {total_time:.2f} seconds")

In [ ]:
# 10. Evaluation on the test set
print("Evaluating model...")
model.load_state_dict(torch.load("../models/best_tag_classifier.pth"))  # Load best model
model.eval()

# Process test data in batches for memory efficiency
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=256)

all_predictions = []
all_labels = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        _, predictions = torch.max(outputs, 1)
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

y_pred = np.array(all_predictions)
y_test_np = np.array(all_labels)

accuracy = accuracy_score(y_test_np, y_pred)
print(f"\nAccuracy: {accuracy:.4f}")

In [ ]:
print("\nClassification Report:")
print(classification_report(
    y_test_np, 
    y_pred,
    labels=np.unique(y_test_np),
    target_names=label_encoder.inverse_transform(np.unique(y_test_np))
))

In [ ]:
# Print feature importances from the model
print("\nAnalyzing feature importance...")
with torch.no_grad():
    weights = model.fc1.weight.cpu().numpy()
    importance = np.abs(weights).mean(axis=0)
    
    # Get feature names (both categorical encoded and continuous)
    cat_feature_names = ohe.get_feature_names_out(categorical_cols)
    all_feature_names = np.concatenate([cat_feature_names, np.array(continuous_cols)])
    
    feature_importance = list(zip(all_feature_names, importance))
    feature_importance.sort(key=lambda x: x[1], reverse=True)
    
    print("\nTop 100 most important features:")
    for feature, imp in feature_importance[:100]:
        print(f"{feature}: {imp:.4f}")